## Check how AROMA edge fraction feature is calculated

In [13]:
import numpy as np
import os
import sys
import subprocess
from past.utils import old_div

# hardcoded paths
tempDir = '/LOCAL/jzerbe/auditory_vs_visual'
#'/LOCAL/jzerbe/faces_vs_houses'
melDir = '/LOCAL/jzerbe/auditory_vs_visual/ds003717/derivatives/melodic/sub-06/sub-06_task-SESS01_space-MNI152NLin2009cAsym_melodic'
#'/LOCAL/jzerbe/faces_vs_houses/ds002938/derivatives/melodic/sub-01/sub-01_task-1back_space-MNI152NLin2009cAsym_melodic'

melIC = os.path.join(melDir, 'melodic_IC.nii.gz')
fslDir = os.path.join(os.environ["FSLDIR"], 'bin', '')

# Get the number of ICs
numICs = int(subprocess.getoutput('%sfslinfo %s | grep dim4 | head -n1 | awk \'{print $2}\'' % (fslDir, melIC) ))
# Loop over ICs
edgeFract = np.zeros(numICs)
csfFract = np.zeros(numICs)

In [15]:
for i in range(0, numICs):
    # Define temporary IC-file
    tempIC = os.path.join(tempDir, 'temp_IC.nii.gz')
    # Extract IC from the merged melodic_IC_thr2MNI2mm file
    os.system(' '.join([os.path.join(fslDir, 'fslroi'),
              melIC,
              tempIC,
              str(i),
              '1']))
    # Change to absolute Z-values
    os.system(' '.join([os.path.join(fslDir, 'fslmaths'),
              tempIC,
              '-abs',
              tempIC]))
    # Get sum of Z-values within the total Z-map (calculate via the mean and number of non-zero voxels)
    totVox = int(subprocess.getoutput(' '.join([os.path.join(fslDir, 'fslstats'),
                                                tempIC,
                                                '-V | awk \'{print $1}\''])))
    totMean = float(subprocess.getoutput(' '.join([os.path.join(fslDir, 'fslstats'),
                                                           tempIC,
                                                           '-M'])))
    totSum = totMean * totVox
    
    # Get sum of Z-values of the voxels located within the CSF (calculate via the mean & number of non-zero voxels)
    csfVox = int(subprocess.getoutput(' '.join([os.path.join(fslDir, 'fslstats'),
                                                #tempIC,
                                                os.path.join('/LOCAL/jzerbe/faces_vs_houses', 'mask_csf.nii.gz'),
                                                #'-k mask_csf.nii.gz',
                                                '-V | awk \'{print $1}\''])))
    csfMean = float(subprocess.getoutput(' '.join([os.path.join(fslDir, 'fslstats'),
                                                     #tempIC,
                                                     os.path.join('/LOCAL/jzerbe/faces_vs_houses', 'mask_csf.nii.gz'),
                                                     #'-k mask_csf.nii.gz',
                                                     '-M'])))
    csfSum = csfMean * csfVox
    
    # Get sum of Z-values of the voxels located within the Edge (calculate via the mean & number of non-zero voxels)
    edgeVox = int(subprocess.getoutput(' '.join([os.path.join(fslDir, 'fslstats'),
                                                     #tempIC,
                                                     os.path.join('/LOCAL/jzerbe/faces_vs_houses', 'mask_edge.nii.gz'),
                                                     #'-k mask_edge.nii.gz',
                                                         '-V | awk \'{print $1}\''])))
                  
    edgeMean = float(subprocess.getoutput(' '.join([os.path.join(fslDir, 'fslstats'),
                                                    #tempIC,
                                                    os.path.join('/LOCAL/jzerbe/faces_vs_houses', 'mask_edge.nii.gz'),
                                                    #'-k mask_edge.nii.gz',
                                                    '-M'])))
    
    edgeSum = edgeMean * edgeVox
    
    # Get sum of Z-values of voxels located outside the brain (calculate via the mean & number of non-zero voxels)
    outVox = int(subprocess.getoutput(' '.join([os.path.join(fslDir, 'fslstats'),
                                                #tempIC,
                                                os.path.join('/LOCAL/jzerbe/faces_vs_houses', 'mask_out.nii.gz'),
                                                #'-k mask_out.nii.gz',
                                                '-V | awk \'{print $1}\''])))
    outMean = float(subprocess.getoutput(' '.join([os.path.join(fslDir, 'fslstats'),
                                                #tempIC,
                                               os.path.join('/LOCAL/jzerbe/faces_vs_houses', 'mask_out.nii.gz'),
                                               #'-k mask_out.nii.gz',
                                               '-M'])))
    outSum = outMean * outVox
    
    # Determine edge and CSF fraction
    if not (totSum == 0):
        edgeFract[i] = old_div((outSum + edgeSum), (totSum - csfSum))
        csfFract[i] = old_div(csfSum, totSum)
    else:
        edgeFract[i] = 0
        csfFract[i] = 0
    
    # Remove the temporary IC-file
    os.remove(tempIC)
    

In [16]:
# auditory
print(edgeFract)

[2.82959133 3.0127958  3.12710619 3.12447829 3.08461726 3.11819121
 3.07502932 3.27055593 2.81412357 3.1861663  3.27442102 3.25365598
 3.31595471 3.3405092  3.064529   3.47356807 3.56380204 3.30866938
 3.2283777  3.22353154 3.44981852 3.28857326 3.50255916 3.20177469
 3.43008522 3.16485991 3.26496965 3.52271785 3.45918102 3.17091407
 3.68168486 3.51455763 3.26195827 3.35611249 3.25984906 3.45539699
 3.48847889 3.4237872  3.43001181 3.3492944  3.33583966]


In [12]:
# faces vs houses
print(edgeFract)

[18.20465255 18.32353867 19.61630835 18.57597024 15.53556241 17.16348447
 20.25773009 20.26236912 20.28871268 22.13152232 23.17118447 18.61446352
 21.54797971 20.16233771 21.18556271 20.09112431 22.92011181 20.91887455
 21.65445202 22.06856774 18.64111113 18.96909265 20.53690474 22.40196567
 21.42811314 21.5305456  22.09052383 23.24822989 22.21180272 19.27785952
 23.03848393 19.79576238 22.99038531 23.18682002 21.07896942 21.50320606
 21.33523855 20.29650806 21.8839725  20.28341059 22.25694748 19.82836805
 20.82216645 22.65970161 22.22399272 21.98409995 21.25551517 23.34064919
 21.9884159  22.1387764  20.41366721 19.40339499 23.48829976 21.99754507
 25.09984394 21.53547148 22.36465094]
